# Bunch Commands

In [ ]:
from pytao import Tao

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Initialize Tao on the CSR beam tracking example

In [ ]:
tao = Tao(
    "-init $ACC_ROOT_DIR/bmad-doc/tao_examples/csr_beam_tracking/tao.init -noplot"
)

## bunch_params

Bunch statistics can be retrieved from any element as a dict.

In [ ]:
stats = tao.bunch_params("end")
stats

This says that the full beam is saved at this element

In [ ]:
stats["beam_saved"]

## bunch1

Array data from a bunch can be retrieved. Available coordinates are:

x, px, y, py, z, pz, s, t, charge, p0c, state, ix_ele

Appropriate data types are returned

In [ ]:
tao.bunch1("end", "x")[0:10]

In [ ]:
tao.bunch1("end", "ix_ele")[0:10]

## Plot in matplotlib

This can be used to plot particles.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%config InlineBackend.figure_format = 'retina' # Nicer plotting
%matplotlib inline

In [ ]:
xdat = tao.bunch1("end", "x")
pxdat = tao.bunch1("end", "px")
chargedat = tao.bunch1("end", "charge")

xdata = 1000 * xdat
ydata = 1000 * pxdat
weights = chargedat

In [ ]:
# hist2d

mycmap = plt.get_cmap("plasma")  # viridis plasma inferno magma and _r versions
mycmap.set_under(color="white")  # map 0 to this color
myvmin = 1e-30  # something tiny
# Bin particles
plt.hist2d(
    x=1000 * xdata, y=ydata, bins=2 * [40], weights=weights, cmap=mycmap, vmin=myvmin
)
plt.xlabel("x (mm)")
plt.ylabel("px (mrad)")
plt.show()

Numpy histogram 2d, with custom color map

In [ ]:
import matplotlib.colors as colors

mycmap = plt.get_cmap("viridis")  # viridis plasma inferno magma and _r versions
mycmap.set_under(color="white")  # map 0 to this color
H, xedges, yedges = np.histogram2d(xdata, ydata, weights=chargedat, bins=40)

xmin, xmax = min(xedges), max(xedges)
ymin, ymax = min(yedges), max(yedges)

image = np.flip(H.T, axis=0)  #
imax = np.max(image)
norm = colors.Normalize(vmin=1e-12 * imax, vmax=imax)
plt.xlabel("x (mm)")
plt.ylabel("px (mrad)")
plt.imshow(
    image, cmap=mycmap, norm=norm, extent=[xmin, xmax, ymin, ymax], aspect="auto"
);

In [ ]:
np.min(image), np.max(image)

## Plot in Bokeh

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh import palettes, colors
from bokeh.models import ColumnDataSource, HoverTool

output_notebook(verbose=False, hide_banner=True)

pal = palettes.Viridis[256]
# white=colors.named.white
# pal[0] = white # replace 0 with white

In [ ]:
H, xedges, yedges = np.histogram2d(xdata, ydata, weights=chargedat, bins=40)
xmin, xmax = min(xedges), max(xedges)
ymin, ymax = min(yedges), max(yedges)

In [ ]:
ds = ColumnDataSource(data=dict(image=[H.transpose()]))
p = figure(
    x_range=[xmin, xmax],
    y_range=[ymin, ymax],
    title="Bunch at end",
    x_axis_label="x (mm)",
    y_axis_label="px (mrad)",
    width=500,
    height=500,
)
p.image(
    image="image",
    source=ds,
    x=xmin,
    y=ymin,
    dw=xmax - xmin,
    dh=ymax - ymin,
    palette=pal,
)
show(p)

## Data for ParticleGroup

The above commands have been packaged into two functions for easier use, and to easily create ParticleGroup objects

In [ ]:
data = tao.bunch_data("end")
data.keys()

In [ ]:
from pmd_beamphysics import ParticleGroup

P = ParticleGroup(data=data)
P

In [ ]:
P.plot("x", "px")

In [ ]:
P.twiss("xy")

# bunch_comb

In [ ]:
tao.bunch_comb("x")

Make a nice plot with the beam envelope

In [ ]:
s = tao.bunch_comb("s")
mean_x = tao.bunch_comb("x")
max_x = mean_x + tao.bunch_comb("rel_max.1")
min_x = mean_x + tao.bunch_comb("rel_min.1")
sigma_x = np.sqrt(tao.bunch_comb("sigma.11"))
fig, ax = plt.subplots()

ax.fill_between(s, min_x, max_x, alpha=0.2)
ax.plot(s, sigma_x, label=r"$+\sigma_x$")
ax.plot(s, mean_x, label=r"$<x>$", marker=".")
ax.plot(s, -sigma_x, label=r"$-\sigma_x$")
ax.set_xlabel("s (m)")
ax.set_ylabel("beam sizes (m)")
plt.legend();

Beam betas

In [ ]:
plt.plot(tao.bunch_comb("s"), 1000 * tao.bunch_comb("x.beta"), label="beam beta_x")
plt.plot(tao.bunch_comb("s"), 1000 * tao.bunch_comb("y.beta"), label="beam beta_y")
plt.xlabel("s (m)")
plt.ylabel("beam Twiss beta (m)")
plt.legend();